In [1]:
# 한국어 감성 분류 - LSTM

# 1. 한글을 입력데이터로 사용하기 위해서는 한글을 숫자로 변환!
#    단어사전(한글단어와 숫자가 1:1로 매칭되는 단어사전) 생성해야 해요!
# 2. 단어를 숫자로 변경하기 위해 문장을 특정한 기준으로 잘라내야 해요!
#    특정한 기준으로 분리한 조각을 token
#    영어는 이 token을 만드는게 어렵지 않아요.
#    공백을 기준으로 잘라서 상대적으로 간단한 처리로 token을 구별해 낼 수 있어요!
#    한글은 이 token을 만드는게 쉽지 않아요. 조사가 있어서 그래요!
#    어쨋든 tokenizer라는 도구를 이용해서 token을 구별할 수 있어요!
# 3. 문장의 길이가 다른 경우 당연히 같은 길이로 만들어 줘야해요!

In [ ]:
%reset -f

from tensorflow.keras.preprocessing.text import Tokenizer

sentences = ['영실이는 나를 정말 정말 좋아해',
             '영실이는 영화를 좋아해']

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

# 공백을 기준으로 token들이 생성되고
# 이 token을 이용해서 vocabulary를 생성해요!
print(tokenizer.word_index) # Vocabulary (단어사전)
# {'영실이는': 1, '정말': 2, '좋아해': 3, '나를': 4, '영화를': 5}

# 단어사전이 만들어졌으니 이를 이용해서
# 문자열을 숫자의 sequence로 변경할 수 있어요!
word_encoding = tokenizer.texts_to_sequences(sentences)
print(word_encoding) # [[1, 4, 2, 2, 3], [1, 5, 3]]

# 만약 단어사전에 없는 새로운 단어가 등장하면 어떻게 되나요?
new_sentence = ['영실이는 거북이와 나를 좋아해']
new_word_encoding = tokenizer.texts_to_sequences(new_sentence)
print(new_word_encoding) # [[1, 4, 3]] Token은 4개지만 encoding된 결과는 3개에요!

{'영실이는': 1, '정말': 2, '좋아해': 3, '나를': 4, '영화를': 5}
[[1, 4, 2, 2, 3], [1, 5, 3]]
[[1, 4, 3]]


In [ ]:
# 이런 경우를 위해 특수한 token을 vocabulary에 등록해 줄 수 있어요!
# OOV(Out of Vocabualry) token을 등록해 줄 수 있어요!
%reset -f

from tensorflow.keras.preprocessing.text import Tokenizer

sentences = ['영실이는 나를 정말 정말 좋아해',
             '영실이는 영화를 좋아해']

tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)
print(tokenizer.word_index)
# {'<OOV>': 1, '영실이는': 2, '정말': 3, '좋아해': 4, '나를': 5, '영화를': 6}

word_encoding = tokenizer.texts_to_sequences(sentences)
print(word_encoding) # [[2, 5, 3, 3, 4], [2, 6, 4]]

new_sentence = ['영실이는 거북이와 나를 좋아해']
new_word_encoding = tokenizer.texts_to_sequences(new_sentence)
print(new_word_encoding) # [[2, 1, 5, 4]]

{'<OOV>': 1, '영실이는': 2, '정말': 3, '좋아해': 4, '나를': 5, '영화를': 6}
[[2, 5, 3, 3, 4], [2, 6, 4]]
[[2, 1, 5, 4]]


In [ ]:
# 추가적으로 문자열 데이터셋에 반도가 적은 단어가 많이 존재하는 경우
# 이런것들은 제외하는게 좋아요!
# 많이 사용하는 단어들이 있고 적게 사용하는 단어가 있어요!
# 빈도수가 많은 단어들만 vocabulary에 포함하는게 좋아요!
# num_words 파라미터를 이용해서 token의 개수를 조절할 수 있어요!
%reset -f

from tensorflow.keras.preprocessing.text import Tokenizer

sentences = ['영실이는 나를 정말 정말 좋아해',
             '영실이는 영화를 좋아해']

tokenizer = Tokenizer(oov_token='<OOV>',
                      num_words=3) # 1, 2, 3번 단어만 사용한다는 의미 (빈도수가 가장 많은 3개 단어)
tokenizer.fit_on_texts(sentences)
print(tokenizer.word_index)
# {'<OOV>': 1, '영실이는': 2, '정말': 3, '좋아해': 4, '나를': 5, '영화를': 6}

word_encoding = tokenizer.texts_to_sequences(sentences)
print(word_encoding) # [[2, 1, 1, 1, 1], [2, 1, 1]]

new_sentence = ['영실이는 거북이와 나를 좋아해']
new_word_encoding = tokenizer.texts_to_sequences(new_sentence)
print(new_word_encoding) # [[2, 1, 1, 1]]

{'<OOV>': 1, '영실이는': 2, '정말': 3, '좋아해': 4, '나를': 5, '영화를': 6}
[[2, 1, 1, 1, 1], [2, 1, 1]]
[[2, 1, 1, 1]]


In [ ]:
# 마지막으로 pad_sequence도 해보아요!
%reset -f

from tensorflow.keras.preprocessing.text import Tokenizer

sentences = ['영실이는 나를 정말 정말 좋아해',
             '영실이는 영화를 좋아해']

tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)
print(tokenizer.word_index)
# {'<OOV>': 1, '영실이는': 2, '정말': 3, '좋아해': 4, '나를': 5, '영화를': 6}

word_encoding = tokenizer.texts_to_sequences(sentences)
print(word_encoding) # [[2, 1, 1, 1, 1], [2, 1, 1]]

# 문자열의 길이를 똑같이 맞춰줘야 해요!
from tensorflow.keras.preprocessing.sequence import pad_sequences

result = pad_sequences(word_encoding,
                       maxlen=4)
print(result) # [[5 3 3 4] [0 2 6 4]] 앞을 자르거나 앞에 0을 붙임.

{'<OOV>': 1, '영실이는': 2, '정말': 3, '좋아해': 4, '나를': 5, '영화를': 6}
[[2, 5, 3, 3, 4], [2, 6, 4]]
[[5 3 3 4]
 [0 2 6 4]]
